# Maldição da Dimensionalidade

A **Maldição da Dimensionalidade** (Curse of Dimensionality) refere-se a vários fenômenos que surgem ao analisarmos e organizarmos dados em espaços de alta dimensão (frequentemente com centenas ou milhares de dimensões) que não ocorrem em espaços de baixa dimensão, como o espaço tridimensional físico que experimentamos no dia a dia.

Neste notebook, exploraremos dois dos efeitos mais contra-intuitivos e impactantes dessa maldição:
1.  O esvaziamento do espaço (a maior parte do volume de um hipercubo concentra-se nos cantos).
2.  A concentração de distâncias (todas as distâncias entre pontos tendem a se tornar iguais).

Esses fenômenos são fundamentais para entender por que algoritmos baseados em distância, como o K-Means e o KNN, falham em altas dimensões e motivam o uso de técnicas de Redução de Dimensionalidade (que veremos nos próximos notebooks).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 1. O Volume da Hiperesfera

Considere uma hiperesfera de raio $r=1$ inscrita em um hipercubo de aresta $2r=2$ (centrado na origem). Em 2D, temos um círculo dentro de um quadrado. Em 3D, uma esfera dentro de um cubo.

Vamos investigar qual fração do volume do hipercubo é ocupada pela hiperesfera à medida que a dimensão $d$ aumenta.

Matematicamente, definimos a razão de volumes como:
$$ \text{Razão} = \frac{\text{Volume da Esfera}_d}{\text{Volume do Cubo}_d} $$

Usaremos uma **Simulação de Monte Carlo** para estimar essa razão:
1. Geramos pontos aleatórios uniformemente distribuídos dentro do hipercubo $[-1, 1]^d$.
2. Contamos quantos desses pontos caem dentro da esfera (distância à origem $\le 1$).
3. A razão de pontos dentro da esfera sobre o total nos dá uma aproximação da razão de volumes.

In [ ]:
def sphere_volume(dim, n_samples=100000):
    # Gerar pontos uniformes no hipercubo [-1, 1]^d
    X = np.random.uniform(-1, 1, size=(n_samples, dim))
    
    # Calcular distância euclidiana ao quadrado (mais rápido) para cada ponto
    # Se sum(coords^2) <= 1, ponto está dentro da esfera unitária
    dist_sq = np.sum(X**2, axis=1)
    points_inside = np.sum(dist_sq <= 1.0)
    
    return points_inside / n_samples

dimesoes_teste = [2, 3, 4, 5, 10, 20]
ratios = []

print(f"{'Dimensão':<10} | {'Razão (Esfera/Cubo)':<25}")
print("-" * 40)

for d in dimesoes_teste:
    ratio = sphere_volume(d)
    ratios.append(ratio)
    print(f"{d:<10} | {ratio:.5f}")

In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(dimesoes_teste, ratios, marker='o', linestyle='-', color='indigo')
plt.title('Fração do Volume do Hipercubo Ocupada pela Hiperesfera')
plt.xlabel('Dimensão ($d$)')
plt.ylabel('Razão Volume Esfera / Volume Cubo')
plt.grid(True)
plt.show()

### Análise

O resultado é surpreendente: em dimensões altas (até mesmo $d=10$ ou $20$), o volume da esfera torna-se insignificante em comparação ao do cubo. Isso significa que **quase todo o volume do espaço de alta dimensão está localizado nos "cantos" do hipercubo**, longe do centro.

Para algoritmos de Machine Learning, isso implica que os dados tornam-se extremamente **esparsos**. Para cobrir o espaço com a mesma densidade de amostras que temos em 1D, a quantidade de dados necessária cresce exponencialmente com a dimensão.

## 2. Concentração de Distâncias

Outro efeito crítico é o comportamento da Distância Euclidiana. À medida que a dimensão aumenta, a diferença proporcional entre a distância do ponto mais próximo e a do ponto mais distante tende a zero.

Sejam $d_{\min}$ e $d_{\max}$ as distâncias mínima e máxima de um ponto de consulta para os outros pontos da amostra. Em altas dimensões:

$$ \lim_{d \to \infty} \frac{d_{\max} - d_{\min}}{d_{\min}} \to 0 $$

Isso significa que, do ponto de vista de qualquer amostra, **todos os outros pontos parecem estar aproximadamente à mesma distância**. O conceito de "vizinho mais próximo" perde o sentido.

In [ ]:
def analisar_distancias(dims, n_points=500):
    plt.figure(figsize=(12, 8))
    
    for i, d in enumerate(dims):
        # Gerar n_points em dimensão d (Distribuição Uniforme [0, 1])
        X = np.random.rand(n_points, d)
        
        # Calcular todas as distâncias par-a-par
        dists = pdist(X, metric='euclidean')
        
        # Calcular métricas
        d_min, d_max, d_mean = np.min(dists), np.max(dists), np.mean(dists)
        contraste = (d_max - d_min) / d_min
        
        # Plotar histograma normalizado pela média para facilitar comparação visual
        # Isso nos mostra a dispersão relativa
        sns.histplot(dists / d_mean, kde=True, label=f'd={d} (Contrast={contraste:.2f})', alpha=0.6, stat='density', element="step")
        
    plt.title('Distribuição das Distâncias Par-a-Par (Normalizadas pela Média)')
    plt.xlabel('Distância / Média')
    plt.ylabel('Densidade')
    plt.legend()
    plt.xlim(0, 3)
    plt.show()

# Dimensões para teste: de baixa (2D) para muito alta (5000D)
dims_to_test = [2, 10, 100, 1000, 5000]
analisar_distancias(dims_to_test)

### Conclusão sobre Distâncias

Observe no gráfico acima como as distribuições se tornam mais estreitas (em relação à média) conforme a dimensão aumenta. 

- Em **2D** (azul), há uma grande variância: alguns pontos estão muito próximos, outros muito distantes.
- Em **5000D** (roxo), o histograma é um pico estreito ao redor da média. A distância para o vizinho mais próximo é quase igual à distância para o vizinho mais distante.

**Impacto no Clustering:** O K-Means, que depende de calcular distâncias para centróides, torna-se ineficaz. O algoritmo converge lentamente ou encontra ótimos locais ruins, pois a distinção espacial entre clusters se dissolve.

## 3. Ortogonalidade Assíntota

Uma consequência menos conhecida, mas igualmente importante, da alta dimensionalidade é que **vetores aleatórios tendem a ser ortogonais entre si**.

Imagine escolher dois vetores aleatórios partindo da origem em um espaço de alta dimensão. A probabilidade de que eles apontem em direções similares (ângulo próximo de 0°) ou opostas (ângulo próximo de 180°) é extremamente baixa. A grande maioria dos pares formará um ângulo muito próximo de 90°.

Isso acontece porque o "equador" de uma hiperesfera concentra a maior parte da área da superfície à medida que a dimensão cresce.

In [ ]:
def analisar_ortogonalidade(dims, n_pairs=10000):
    plt.figure(figsize=(10, 6))
    
    for d in dims:
        # Gerar pares de vetores aleatórios (distribuição normal funciona bem para direção isotrópica)
        u = np.random.randn(n_pairs, d)
        v = np.random.randn(n_pairs, d)
        
        # Calcular cosseno do ângulo: (u . v) / (|u| |v|)
        dot_product = np.sum(u * v, axis=1)
        norm_u = np.linalg.norm(u, axis=1)
        norm_v = np.linalg.norm(v, axis=1)
        
        # Evitar divisão por zero
        valid = (norm_u > 0) & (norm_v > 0)
        
        cos_theta = dot_product[valid] / (norm_u[valid] * norm_v[valid])
        
        # Garantir que valores estejam em [-1, 1] para evitar erro numérico no arccos
        cos_theta = np.clip(cos_theta, -1.0, 1.0)
        
        # Converter para graus
        angles = np.degrees(np.arccos(cos_theta))
        
        sns.histplot(angles, kde=True, label=f'd={d}', alpha=0.6, element="step", stat='density')
        
    plt.title('Distribuição dos Ângulos entre Vetores Aleatórios')
    plt.xlabel('Ângulo (graus)')
    plt.ylabel('Densidade')
    plt.axvline(90, color='red', linestyle='--', alpha=0.5, label='90°')
    plt.legend()
    plt.xlim(0, 180)
    plt.show()

analisar_ortogonalidade([2, 10, 100, 1000])

### Análise da Ortogonalidade

- Em **2D**, os ângulos estão uniformemente espalhados.
- Conforme **d aumenta**, a distribuição se concentra agudamente em torno de **90 graus**.

Isso tem implicações profundas: em Espaços Vetoriais de Alta Dimensão (como em Embeddings de palavras de 768 ou 1536 dimensões), quase todos os vetores são ortogonais entre si. Isso pode ser vantajoso (capacidade de armazenar muitas informações distintas quase ortogonalmente) ou problemático (dificuldade em encontrar similaridades significativas se não houver estrutura nos dados).

## 4. Como lidar com a Maldição?

Para combater esses efeitos em datasets de alta dimensão (como imagens, embeddings de texto, dados genômicos), utilizamos técnicas de **Redução de Dimensionalidade**.

O objetivo é encontrar um subespaço de menor dimensão (manifold) onde os dados "vivem" intrinsecamente, preservando suas estruturas importantes.

Nos próximos notebooks, estudaremos as principais técnicas para realizar essa tarefa:
1.  **PCA (Análise de Componentes Principais)**: Redução linear que preserva variância global.
2.  **t-SNE**: Redução não-linear focada em preservar vizinhanças locais (ótima para visualização).
3.  **UMAP**: Alternativa moderna ao t-SNE que equilibra estrutura local e global.